# Construir um modelo de regressão: preparar e visualizar dados

## **Regressão Linear para Abóboras - Aula 2**
#### Introdução

Agora que você já está equipado com as ferramentas necessárias para começar a construir modelos de aprendizado de máquina com Tidymodels e o Tidyverse, está pronto para começar a fazer perguntas sobre seus dados. Ao trabalhar com dados e aplicar soluções de aprendizado de máquina, é muito importante entender como formular a pergunta certa para desbloquear adequadamente o potencial do seu conjunto de dados.

Nesta aula, você aprenderá:

-   Como preparar seus dados para a construção de modelos.

-   Como usar `ggplot2` para visualização de dados.

A pergunta que você precisa responder determinará o tipo de algoritmos de aprendizado de máquina que você utilizará. E a qualidade da resposta que você obtiver dependerá fortemente da natureza dos seus dados.

Vamos ver isso na prática por meio de um exercício.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Arte por @allison_horst</figcaption>


## 1. Importando dados de abóboras e invocando o Tidyverse

Vamos precisar dos seguintes pacotes para explorar e manipular esta lição:

-   `tidyverse`: O [tidyverse](https://www.tidyverse.org/) é uma [coleção de pacotes R](https://www.tidyverse.org/packages) projetada para tornar a ciência de dados mais rápida, fácil e divertida!

Você pode instalá-los assim:

`install.packages(c("tidyverse"))`

O script abaixo verifica se você possui os pacotes necessários para completar este módulo e os instala para você caso algum esteja faltando.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Agora, vamos iniciar alguns pacotes e carregar os [dados](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) fornecidos para esta lição!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Um rápido `glimpse()` mostra imediatamente que há valores em branco e uma mistura de strings (`chr`) e dados numéricos (`dbl`). A coluna `Date` está no formato de texto e há também uma coluna estranha chamada `Package`, onde os dados são uma mistura de `sacks`, `bins` e outros valores. Na verdade, os dados estão um pouco bagunçados 😤.

De fato, não é muito comum receber um conjunto de dados completamente pronto para ser usado na criação de um modelo de ML diretamente. Mas não se preocupe, nesta lição, você aprenderá como preparar um conjunto de dados bruto usando bibliotecas padrão do R 🧑‍🔧. Você também aprenderá várias técnicas para visualizar os dados. 📈📊
<br>

> Um lembrete: O operador pipe (`%>%`) realiza operações em sequência lógica, passando um objeto adiante para uma função ou expressão. Você pode pensar no operador pipe como se estivesse dizendo "e então" no seu código.


## 2. Verificar dados ausentes

Um dos problemas mais comuns que os cientistas de dados precisam lidar é com dados incompletos ou ausentes. O R representa valores ausentes ou desconhecidos com um valor especial chamado sentinela: `NA` (Not Available).

Então, como saberíamos que o data frame contém valores ausentes?  
<br>
-   Uma maneira direta seria usar a função base do R `anyNA`, que retorna os objetos lógicos `TRUE` ou `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Ótimo, parece que há alguns dados faltando! Esse é um bom ponto de partida.

-   Outra maneira seria usar a função `is.na()`, que indica quais elementos individuais da coluna estão ausentes com um valor lógico `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Certo, trabalho concluído, mas com um dataframe tão grande como este, seria ineficiente e praticamente impossível revisar todas as linhas e colunas individualmente😴.

-   Uma maneira mais intuitiva seria calcular a soma dos valores ausentes para cada coluna:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Muito melhor! Há dados faltando, mas talvez isso não importe para a tarefa em questão. Vamos ver o que uma análise mais aprofundada revela.

> Junto com os incríveis conjuntos de pacotes e funções, o R possui uma documentação muito boa. Por exemplo, use `help(colSums)` ou `?colSums` para saber mais sobre a função.


## 3. Dplyr: Uma Gramática para Manipulação de Dados

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Arte por @allison_horst</figcaption>


<!--![Arte por \@allison_horst](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.br.png)<br/>Arte por \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), um pacote do Tidyverse, é uma gramática de manipulação de dados que oferece um conjunto consistente de verbos para ajudar a resolver os desafios mais comuns de manipulação de dados. Nesta seção, vamos explorar alguns dos verbos do dplyr!  
<br>


#### dplyr::select()

`select()` é uma função do pacote `dplyr` que ajuda você a escolher colunas para manter ou excluir.

Para tornar seu data frame mais fácil de trabalhar, remova várias de suas colunas usando `select()`, mantendo apenas as colunas que você precisa.

Por exemplo, neste exercício, nossa análise envolverá as colunas `Package`, `Low Price`, `High Price` e `Date`. Vamos selecionar essas colunas.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` é uma função do pacote `dplyr` que ajuda a criar ou modificar colunas, mantendo as colunas existentes.

A estrutura geral do `mutate` é:

`data %>%   mutate(nome_da_nova_coluna = o_que_ela_contem)`

Vamos experimentar o `mutate` usando a coluna `Date` e realizando as seguintes operações:

1. Converter as datas (atualmente do tipo caractere) para um formato de mês (essas são datas no formato dos EUA, ou seja, `MM/DD/YYYY`).

2. Extrair o mês das datas para uma nova coluna.

No R, o pacote [lubridate](https://lubridate.tidyverse.org/) facilita o trabalho com dados de data e hora. Então, vamos usar `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` e ver como alcançar os objetivos acima. Podemos descartar a coluna `Date`, já que não precisaremos mais dela nas operações subsequentes.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Uhuu! 🤩

Agora, vamos criar uma nova coluna chamada `Price`, que representa o preço médio de uma abóbora. Para isso, vamos calcular a média das colunas `Low Price` e `High Price` para preencher a nova coluna Price.  
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Yeees!💪

"Mas espere!", você dirá depois de dar uma olhada rápida no conjunto de dados com `View(pumpkins)`, "Tem algo estranho aqui!"🤔

Se você observar a coluna `Package`, verá que as abóboras são vendidas em várias configurações diferentes. Algumas são vendidas em medidas de `1 1/9 bushel`, outras em medidas de `1/2 bushel`, algumas por abóbora, outras por libra, e algumas em grandes caixas com larguras variadas.

Vamos verificar isso:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Incrível!👏

Parece que abóboras são muito difíceis de pesar de forma consistente, então vamos filtrá-las selecionando apenas as abóboras que contêm a string *bushel* na coluna `Package` e colocar isso em um novo data frame `new_pumpkins`.


#### dplyr::filter() e stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): cria um subconjunto dos dados contendo apenas as **linhas** que atendem às suas condições, neste caso, abóboras com a string *bushel* na coluna `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): detecta a presença ou ausência de um padrão em uma string.

O pacote [`stringr`](https://github.com/tidyverse/stringr) fornece funções simples para operações comuns com strings.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Você pode ver que reduzimos para cerca de 415 linhas de dados contendo abóboras por alqueire. 🤩


#### dplyr::case_when()

**Mas espere! Ainda há mais uma coisa a fazer**

Você percebeu que a quantidade de alqueires varia por linha? É necessário normalizar os preços para que sejam exibidos por alqueire, e não por 1 1/9 ou 1/2 alqueire. Hora de fazer alguns cálculos para padronizar isso.

Usaremos a função [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) para *modificar* a coluna Price dependendo de algumas condições. `case_when` permite que você vetorize múltiplas declarações `if_else()`.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Agora, podemos analisar o preço por unidade com base na medida de alqueire. Todo esse estudo sobre alqueires de abóboras, no entanto, mostra como é `importante` `entender a natureza dos seus dados`!

> ✅ De acordo com [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), o peso de um alqueire depende do tipo de produto, já que é uma medida de volume. "Um alqueire de tomates, por exemplo, deve pesar 56 libras... Folhas e verduras ocupam mais espaço com menos peso, então um alqueire de espinafre pesa apenas 20 libras." É tudo bem complicado! Vamos evitar fazer uma conversão de alqueire para libra e, em vez disso, precificar pelo alqueire. Todo esse estudo sobre alqueires de abóboras, no entanto, mostra como é muito importante entender a natureza dos seus dados!
>
> ✅ Você percebeu que as abóboras vendidas por meio alqueire são muito caras? Consegue descobrir o motivo? Dica: abóboras pequenas são muito mais caras do que as grandes, provavelmente porque há muito mais delas por alqueire, considerando o espaço não utilizado ocupado por uma grande abóbora oca de torta.


Agora, por pura diversão 💁‍♀️, vamos também mover a coluna Mês para a primeira posição, ou seja, `antes` da coluna `Pacote`.

`dplyr::relocate()` é usado para alterar as posições das colunas.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Bom trabalho!👌 Agora você tem um conjunto de dados limpo e organizado para construir seu novo modelo de regressão!  
<br>


## 4. Visualização de dados com ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infográfico por Dasani Madipalli</figcaption>


<!--![Infográfico por Dasani Madipalli](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.br.png){width="600"}-->

Existe um *sábio* ditado que diz o seguinte:

> "O gráfico simples trouxe mais informações à mente do analista de dados do que qualquer outro dispositivo." --- John Tukey

Parte do papel do cientista de dados é demonstrar a qualidade e a natureza dos dados com os quais está trabalhando. Para isso, eles frequentemente criam visualizações interessantes, como gráficos, diagramas e tabelas, mostrando diferentes aspectos dos dados. Dessa forma, eles conseguem mostrar visualmente relações e lacunas que, de outra maneira, seriam difíceis de identificar.

As visualizações também podem ajudar a determinar a técnica de aprendizado de máquina mais apropriada para os dados. Um gráfico de dispersão que parece seguir uma linha, por exemplo, indica que os dados são bons candidatos para um exercício de regressão linear.

O R oferece vários sistemas para criar gráficos, mas [`ggplot2`](https://ggplot2.tidyverse.org/index.html) é um dos mais elegantes e versáteis. O `ggplot2` permite compor gráficos **combinando componentes independentes**.

Vamos começar com um gráfico de dispersão simples para as colunas Price e Month.

Neste caso, começaremos com [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), fornecendo um conjunto de dados e um mapeamento estético (com [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)) e, em seguida, adicionaremos camadas (como [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) para gráficos de dispersão.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Este é um gráfico útil 🤷? Algo nele te surpreende?

Não é particularmente útil, já que tudo o que ele faz é mostrar seus dados como uma distribuição de pontos em um determinado mês.
<br>


### **Como tornamos isso útil?**

Para exibir dados úteis em gráficos, geralmente é necessário agrupar os dados de alguma forma. Por exemplo, no nosso caso, calcular o preço médio das abóboras para cada mês forneceria mais insights sobre os padrões subjacentes em nossos dados. Isso nos leva a mais uma abordagem do **dplyr**:

#### `dplyr::group_by() %>% summarize()`

A agregação agrupada no R pode ser facilmente realizada usando

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` altera a unidade de análise do conjunto de dados completo para grupos individuais, como por mês.

-   `dplyr::summarize()` cria um novo data frame com uma coluna para cada variável de agrupamento e uma coluna para cada estatística resumida que você especificou.

Por exemplo, podemos usar `dplyr::group_by() %>% summarize()` para agrupar as abóboras com base na coluna **Month** e, em seguida, calcular o **preço médio** para cada mês.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Sucinto!✨

Características categóricas, como meses, são melhor representadas usando um gráfico de barras 📊. As camadas responsáveis por gráficos de barras são `geom_bar()` e `geom_col()`. Consulte `?geom_bar` para saber mais.

Vamos criar um agora!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 Esta é uma visualização de dados mais útil! Parece indicar que o preço mais alto para abóboras ocorre em setembro e outubro. Isso atende às suas expectativas? Por que sim ou por que não?

Parabéns por concluir a segunda lição 👏! Você preparou seus dados para a construção do modelo e, em seguida, descobriu mais insights usando visualizações!



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte oficial. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
